<a href="https://colab.research.google.com/github/TheLastGimbus/Roll-API-Tensorflow-Training/blob/master/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's import stuff!

In [0]:
%tensorflow_version 2.x

import numpy as np
import tensorflow as tf
